In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import defaultdict, Counter

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pathlib import Path
import re
import torch
from typing import Union, Callable
from tqdm.auto import tqdm

from src.utils import concat_csv_with_indices

In [ ]:
sns.set_context("talk", font_scale=1.5)

In [ ]:
state_space_spec_path = "outputs/state_space_specs/librispeech-train-clean-100/w2v2_8/state_space_specs.pkl"



In [ ]:
predictions_df = concat_csv_with_indices("outputs/word_recognition/librispeech-train-clean-100/**/predictions-frame_*.csv",
                        [re.compile(r"frame_(\d+)\.csv"),
                         lambda p: p.parents[2].name,
                         lambda p: p.parents[3].name, lambda p: p.parents[4].name,
                         lambda p: p.parents[5].name],
                        ["frame_idx",
                         "equivalence", "model", "base_model", "dataset"
                          ]).reset_index()

predictions_df["frame_idx"] = predictions_df.frame_idx.astype(int)

predictions_df["trace"] = predictions_df.base_model + "/" + predictions_df.model + "/" + predictions_df.equivalence

In [ ]:
plot_traces = {
    "Word": "w2v2_8/rnn_32-hinge-mAP4/word_broad_10frames",
    "Random": "w2v2_8/randomrnn_32/random",
    "Phoneme": "w2v2_8/rnn_8-weightdecay0.01/phoneme_10frames",
}
plot_max_num_frames = 9
predictions_df = predictions_df[predictions_df.trace.isin(plot_traces.values())]
predictions_df["Trace"] = predictions_df.trace.map({v: k for k, v in plot_traces.items()})
predictions_df["# phonemes seen"] = predictions_df.frame_idx + 1

In [ ]:
# estimate chance performance for word recognition at each dataset + frame
chance_performance = {}
for dataset, dataset_results in predictions_df.groupby("dataset"):
    # take any trace; they're all the same
    chance_df = next(iter(dataset_results.groupby("trace")))[1]
    # # words should all be balanced in this evaluaton, so we can take the mean of relative word frequency
    # words may not be balanced; take the max-probability word assuming random guessing
    chance_performance[dataset] = chance_df.groupby("frame_idx").apply(lambda x: x.label.value_counts(normalize=True).max()).rename("chance_performance")
chance_performance = pd.concat(chance_performance, names=["dataset"])
chance_performance

In [ ]:
acc_df = predictions_df.loc[(predictions_df.dataset == plot_dataset)] \
    .groupby(["Trace", "# phonemes seen"]).correct.mean().reset_index()
hue_order = acc_df.groupby("Trace").correct.mean().sort_values(ascending=False).index

acc_df = acc_df[acc_df["# phonemes seen"] <= plot_max_num_frames]

f, ax = plt.subplots(figsize=(10, 6))
ax = sns.lineplot(data=acc_df,
                  x="# phonemes seen", y="correct", hue="Trace", hue_order=hue_order,
                  ax=ax)
ax.set_xticks(range(1, plot_max_num_frames + 1))

ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
ax.set_xlabel("# phonemes seen")
ax.set_ylabel("Accuracy")

# draw chance performance as a horizontal line over each x value
for frame, frame_chance_perf in chance_performance.loc[plot_dataset].items():
    if frame + 2 > plot_max_num_frames:
        continue
    ax.plot([frame + 1, frame + 2], [frame_chance_perf, frame_chance_perf],
            color="gray", linestyle="--")

In [ ]:
acc_df.groupby(["Trace"]).correct.max()

## Frequency effects in recognition

In [ ]:
word_freq_df = pd.read_csv("data/SUBTLEXus74286wordstextversion.txt", sep="\t", index_col=0)

In [ ]:
num_frequency_bins = 6
word_freq_df["log_frequency_bin"] = pd.qcut(word_freq_df.Lg10WF, num_frequency_bins, labels=list(range(num_frequency_bins - 1)), duplicates="drop").astype(int)

In [ ]:
predictions_df_meta = pd.merge(predictions_df, word_freq_df, left_on="label", right_index=True)

In [ ]:
ax = sns.lineplot(data=predictions_df_meta[(predictions_df_meta.Trace == "Word") &
                                           (predictions_df_meta["# phonemes seen"] <= plot_max_num_frames)],
                  x="# phonemes seen", y="correct", hue="log_frequency_bin", palette="viridis")
ax.set_xticks(range(1, plot_max_num_frames + 1))
ax.legend(title="Word frequency bin", loc="upper left", bbox_to_anchor=(1, 1))

## Accuracy distribution between words

In [ ]:
accuracy_distribution_df = predictions_df_meta \
    .groupby(["Trace", "label", "label_instance_idx", "Lg10WF", "log_frequency_bin"]).correct.max() \
    .groupby(["Trace", "label", "Lg10WF", "log_frequency_bin"]).mean()
accuracy_distribution_df

In [ ]:
col_order = accuracy_distribution_df.groupby("Trace").mean().sort_values(ascending=False).index
sns.displot(data=accuracy_distribution_df.reset_index(), 
            x="correct", col="Trace", col_order=col_order, col_wrap=3,
            facet_kws=dict(sharex=False), height=4, aspect=1.5)

In [ ]:
g = sns.relplot(data=accuracy_distribution_df.reset_index(),
            x="log_frequency_bin", y="correct",
            hue="Trace", hue_order=accuracy_distribution_df.groupby("Trace").mean().sort_values(ascending=False).index,
            kind="line", aspect=1.5)
g.axes.flat[0].set_xlabel("Word frequency bin")